In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service

In [2]:
user_name = os.environ.get('USER')
password = os.environ.get('PASSWORD')
login_url = os.environ.get('URL')

In [25]:
s = Service("geckodriver.exe")
opts=Options()
opts.add_argument('-headless')
browser = webdriver.Firefox(service=s)
browser.get(login_url)

In [26]:
log_email = browser.find_element(By.ID, value="ap_email")
log_pwd = browser.find_element(By.ID, value="ap_password")
log_email.send_keys(user_name)
log_pwd.send_keys(password)
log_pwd.submit()

In [27]:
browser.get('http://www.goodreads.com/search?q=&qid=')
search_book = browser.find_element(By.ID, value='search_query_main')
search_book.send_keys("1984")
search_book.submit()

In [28]:
itemqueue = browser.find_elements(By.XPATH, value="//table/tbody/tr[contains(@itemtype, 'http://schema.org/Book')]")
img = browser.find_elements(By.CLASS_NAME, value="bookCover")
book_list = list()
for i in range(len(itemqueue)):
    book_list.append(itemqueue[i].text.split('\n'))
    book_list_ap = list()
for i in range(0, len(book_list)):
    book_list_ap.append((book_list[i][0],book_list[i][1],book_list[i][2],img[i].get_property("src")))

In [29]:
itemqueue

[<selenium.webdriver.remote.webelement.WebElement (session="01a9cfa6-0674-4174-bf71-bc04881aa7be", element="bdfcc445-d7a9-438e-a2a7-3b5239229e02")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01a9cfa6-0674-4174-bf71-bc04881aa7be", element="70c1a3f1-0149-4c0c-adbb-f2be028dce56")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01a9cfa6-0674-4174-bf71-bc04881aa7be", element="00512ab0-a4d1-4ba1-a5a2-d91291ad94e6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01a9cfa6-0674-4174-bf71-bc04881aa7be", element="d72092ce-c87b-4abc-98e0-95eb0b52049f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01a9cfa6-0674-4174-bf71-bc04881aa7be", element="996b66f5-7e03-42e8-8090-9e2e9666254a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01a9cfa6-0674-4174-bf71-bc04881aa7be", element="64ac1869-ab91-47a4-8923-a5cfe80ed7cf")>,
 <selenium.webdriver.remote.webelement.WebElement (session="01a9cfa6-0674-4174-bf71-bc04881aa7be", element

In [8]:
def find_book(title, author):
    # Searches the title
    browser.get('http://www.goodreads.com/search?q=&qid=')
    search_book = browser.find_element(By.ID, value='search_query_main')
    search_book.send_keys(title)
    search_book.submit()
    
    # Finds book in the title that's by the author
    itemqueue = browser.find_elements(By.XPATH, value="//table/tbody/tr[contains(@itemtype, 'http://schema.org/Book')]")
    img = browser.find_elements(By.CLASS_NAME, value="bookCover")
    book_list = list()
    for i in range(len(itemqueue)):
        info = itemqueue[i]
        if 'by ' + author in info[1]:
            break
    return info
        


In [24]:
browser.quit()